In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

import os
import re
import glob


In [78]:
# Read data
crop_coordinate = pd.read_csv("data/Crop_Coordinates.csv")

# Hyper preameter 
H_radius, V_radius = 20, 42

# Horizontal margin, meaning how far each champion stay apart from each other on the same row
# S = self, O = oppoinent
H_margin_S, V_margin_S = 52, 34
H_margin_O, V_margin_O = 48.5, 25

# Grow of margin, from top to down
H_margin_Grow, V_margin_Grow = 0.025, 0

In [79]:
coordinate_gen_S = np.zeros((4, 7, 2))
coordinate_gen_O = np.zeros((4, 7, 2))

coordinate_gen_O[0,0,:] = (260, 70)
coordinate_gen_O[1,0,:] = (282, 70 + V_margin_O)
coordinate_gen_O[2,0,:] = (257, 70 + V_margin_O * (1 + V_margin_Grow)**1 * 2)
coordinate_gen_O[3,0,:] = (277, 70 + V_margin_O * (1 + V_margin_Grow)**2 * 3)

coordinate_gen_S[0,0,:] = (248, 176)
coordinate_gen_S[1,0,:] = (277, 176 + V_margin_S)
coordinate_gen_S[2,0,:] = (249, 176 + V_margin_S * (1 + V_margin_Grow)**1 * 2)
coordinate_gen_S[3,0,:] = (255, 176 + V_margin_S * (1 + V_margin_Grow)**2 * 3)

for i in range(4):
    H_margin_S_curr = H_margin_S * (1 + H_margin_Grow) ** i
    H_margin_O_curr = H_margin_O * (1 + H_margin_Grow) ** i
    coordinate_gen_S[i,:,0] = np.arange(coordinate_gen_S[i,0,0], 
                                        coordinate_gen_S[i,0,0] + H_margin_S_curr * 6 + 1, H_margin_S_curr)
    coordinate_gen_O[i,:,0] = np.arange(coordinate_gen_O[i,0,0], 
                                        coordinate_gen_O[i,0,0] + H_margin_O_curr * 6 + 1, H_margin_O_curr)
    coordinate_gen_S[i,:,1] = coordinate_gen_S[i,0,1]
    coordinate_gen_O[i,:,1] = coordinate_gen_O[i,0,1]

coordinate_gen_O = np.round(coordinate_gen_O, 0)
coordinate_gen_S = np.round(coordinate_gen_S, 0)

In [80]:
coordinate_gen_S_modeled = pd.DataFrame(data=coordinate_gen_S.reshape(4*7, -1),
             columns=["X", "Y"])
coordinate_gen_O_modeled = pd.DataFrame(data=coordinate_gen_O.reshape(4*7, -1),
             columns=["X", "Y"])

In [81]:
#coordinate_gen_O_modeled

### Crop/Boundary Images

In [82]:
def crop_all_images(folder_name, specific_list = []):
    """
    Folder structure should be:
    Master_FolderName:
        |___YouTuber_Name:
                |___ Video_Name
                        |___ frame_num.jpg

    folder_path: path to master_folder containing structure defined above
    specific_list: name of YouTubers just added (so you don't have to run everything)

    output:
        crops all .jpg files in folder_path     
        
    """

    folder_path = os.path.join(os.getcwd(), folder_name)
    print(folder_path)

    # Loop through all or specific YouTubers:
    if specific_list:
        youtuber_path_list = []
        for youtuber_name in specific_list:
            youtuber_path_list.append(os.path.join(folder_path, youtuber_name))
    else:
        youtuber_path_list = os.listdir(os.path.join(folder_path))

    for youtuber_name in youtuber_path_list:
        print(youtuber_name)

        youtuber_path = os.path.join(folder_path, youtuber_name)
        
        # Loop through all Videos
        for video_name in os.listdir(os.path.join(youtuber_path)):
            print(video_name)
            video_path = os.path.join(youtuber_path, video_name)
            
            # Loop through all Frames (only the .jpg files and not the folders)
            for img_path in glob.glob(os.path.join(video_path, '*.jpg')):

                # Crop single image
                crop_image(img_path, coordinate_gen_S_modeled, coordinate_gen_O_modeled)

    pass


def test_crop_boundary(img_path, coordinate):

    # Display bounding box

    with Image.open(img_path) as im:
        
        draw = ImageDraw.Draw(im)
        
        for index, row in coordinate.iterrows():
            X, Y = row[0], row[1] 
            #draw.rectangle(((X - H_radius, Y - V_radius), (X + H_radius, Y + V_radius)), outline="red")
            draw.ellipse(((X - 5, Y - 5), (X + 5, Y + 5)), fill = 'red', outline ='red')
            
        im.show()
    pass


def crop_image(img_path, coordinate_S, coordinate_O):
    # Cropped image into chunks

    cropped_img_folder_path = re.match(r'.+(?=\.jpg)', img_path)[0]
    # Create directory of cropped_images if it doesn't exist
    if not os.path.exists(cropped_img_folder_path):
        os.makedirs(cropped_img_folder_path)
    
    with Image.open(img_path) as im:

        for index, row in coordinate_S.iterrows():
            X, Y = row[0], row[1]
            cropper = (X - H_radius, Y - V_radius, X + H_radius, Y + V_radius)

            #path of crop will be: img_path/cropped_S_XXX.png
            save_path = os.path.join(cropped_img_folder_path,"cropped_S_%d.png" % index)
            
            im.crop(cropper).save(save_path, "PNG")

        for index, row in coordinate_O.iterrows():
            X, Y = row[0], row[1]
            cropper = (X - H_radius, Y - V_radius, X + H_radius, Y + V_radius)

            #path of crop will be: img_path/cropped_O_XXX.png
            save_path = os.path.join(cropped_img_folder_path,"cropped_O_%d.png" % index)
            
            im.crop(cropper).save(save_path, "PNG")

    pass

In [84]:
crop_all_images('contents', ['K3soju'])

/Users/mgw/Desktop/CS686_Project/teamfight-tactics-ai/contents
/Users/mgw/Desktop/CS686_Project/teamfight-tactics-ai/contents/K3soju
3 STAR SQUID GAMER + 4 SPELLWEAVER FIDDLESTICKS!!  Teamfight Tactics Patch 1119.mp4
5 SYNDICATE AKALI CARRY SHREDS ENTIRE ENEMY TEAM 1V9!!  Teamfight Tactics Patch 1124.mp4
SILCO AND 7 INNOVATOR DRAGON BUFFS ARE OP!!  Teamfight Tactics Patch 125.mp4
JINX AND JAYCE DUO LEGENDARY CARRIES WITH GIGA-HIGHROLL SCRAP OPENER!!  Teamfight Tactics SET 6 PBE.mp4
6 SCHOLAR YUUMI INFINITE MANA TO ULT STUN AND SHIELD WHOLE TEAM!!  Teamfight Tactics Patch 1124.mp4
SNIPER AHRI CARRY WITH 7 SYNDICATE RESISTANCES!!  Teamfight Tactics Patch 124.mp4
8 CHALLENGER KAISA CARRY WITH RICH GET RICHER FAST 9!!  Teamfight Tactics Patch 1123.mp4
7 MUTANT VOIDBORNE EXECUTE WITH 80 TRUE DAMAGE BUFF!!  Teamfight Tactics Patch 124.mp4
TFT SET 65 CHALLENGER SERIES DAY 1 5 INNOVATORS ARE OP!!  Teamfight Tactics Patch 125B.mp4
DOUBLE DEATHCAP TEEMO CARRY + RADIANT HOJ + RADIANT CHALICE BUFF

In [ ]:
# # Display bounding box
# img_path = "img/11/frame36630.jpg"

# with Image.open(img_path) as im:
    
#     draw = ImageDraw.Draw(im)
    
#     for index, row in coordinate_gen_S_modeled.iterrows():
#         X, Y = row[0], row[1] 
#         #draw.rectangle(((X - H_radius, Y - V_radius), (X + H_radius, Y + V_radius)), outline="red")
#         draw.ellipse(((X - 5, Y - 5), (X + 5, Y + 5)), fill = 'red', outline ='red')
        
#     im.show()

In [ ]:
# # Cropped image into chunks
# img_path = "img/11/frame36630.jpg"

# with Image.open(img_path) as im:
#     for index, row in coordinate_gen_S_modeled.iterrows():
#         X, Y = row[0], row[1]
#         cropper = (X - H_radius, Y - V_radius, X + H_radius, Y + V_radius)
        
#         im.crop(cropper).save("img/11/frame18900/test_cropped%d.png" % index, "PNG")

In [ ]:
if [1,2,3]:
    print('what')